# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы изучите методы работы с категориальными переменными

In [66]:
%pylab inline
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split



%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(2 балла)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [67]:
import pandas as pd
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [68]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

np.random.seed(910)

def create_balanced_dataset(data, target, samples_per_class=100000):
    class_1_idx = np.where(target == 1)[0]
    class_0_idx = np.where(target == 0)[0]
    
    selected_1 = np.random.choice(class_1_idx, samples_per_class, replace=True)
    selected_0 = np.random.choice(class_0_idx, samples_per_class, replace=True)
    
    combined_data = pd.concat([data.iloc[selected_1], data.iloc[selected_0]])
    combined_target = np.hstack([target[selected_1], target[selected_0]])
    
    return combined_data, combined_target

data, target = create_balanced_dataset(data, target)

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [69]:
from sklearn.preprocessing import StandardScaler

def normalize_features(df):
    standard_scaler = StandardScaler()
    normalized_array = standard_scaler.fit_transform(df)
    return pd.DataFrame(normalized_array, columns=df.columns)

scaled_features = normalize_features(data)
scaled_features.describe()
data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,1.366550,4.483870,0.429490,0.502265,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.674421,2.739255,0.496689,1.501934,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,7.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию, найдтие оптимум. Сделайте предсказание на тестовой части выборки. Замерьте качество.

In [70]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

def evaluate_model(X_tr, y_tr, X_te, y_te):
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_tr, y_tr)
    predictions = lr_model.predict(X_te)
    acc = accuracy_score(y_te, predictions)
    prec = precision_score(y_te, predictions)
    rec = recall_score(y_te, predictions)
    f1 = f1_score(y_te, predictions)
    return [acc, prec, rec, f1]

def display_metrics(metrics):
    print(f"Accuracy: {metrics[0]:.4f}")
    print(f"Precision: {metrics[1]:.4f}")
    print(f"Recall: {metrics[2]:.4f}")
    print(f"F1: {metrics[3]:.4f}")

X_train, X_test, y_train, y_test = train_test_split(normalized_data, target, test_size=0.7)

display_metrics(evaluate_model(X_train, y_train, X_test, y_test))

Accuracy: 0.5890
Precision: 0.5981
Recall: 0.5460
F1: 0.5709


__Выводы__ в свободной форме:

Модель LogisticRegression (логистической регрессии) уже из коробки предоставляет нам довольно неплохие результаты. Остается правильно отрегулировать и правильно настроить, чтобы повысить эффективность модели.

__Задание 2.__ Изучение влияния регуляризатора на процесс обучения

__(2 балла)__

Проверьте на практике, как влияет регуляризатор на процесс обучения (убывание функции потерь на обучающей и отложенной выборках). Чтобы считать функцию потерь на отложенной выборке после каждой итерации, запускайте процесс обучения логистической регрессии с параметром $max\_iter=1$ и $w^{(0)}$, полученным на предыдущей итерации. Постройте два графика: на одном из них логистическая регрессия с коэффициентом регуляризации, равным 0, а на другом с некоторым разумным значением. На каждом графике одновременно должна быть и функция потерь для обучающей, и для тестовой выборки. Не забудьте сделать одинаковыми оси обоих графиков. Какие выводы вы можете сделать?

In [71]:
# Your code here
# ...fds

__Выводы:__

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [72]:
from sklearn.preprocessing import OneHotEncoder

cat_columns = [col for col in normalized_data.columns if col.endswith('_cat')]

one_hot_encoder = OneHotEncoder(sparse_output=False, drop="first")
one_hot_encoded = one_hot_encoder.fit_transform(normalized_data[cat_columns])

data_encoded = normalized_data.drop(cat_columns, axis=1)
one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(cat_columns))

data_encoded.reset_index(drop=True, inplace=True)
data_encoded = pd.concat([data_encoded, one_hot_df], axis=1)

X_train, X_test, y_train, y_test = train_test_split(normalize(data_encoded), target, test_size=0.5)
data_encoded.describe()

,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,...,ps_car_11_cat_0.9698844814339967,ps_car_11_cat_0.9999078260160954,ps_car_11_cat_1.0299311705981942,ps_car_11_cat_1.0599545151802927,ps_car_11_cat_1.0899778597623915,ps_car_11_cat_1.1200012043444902,ps_car_11_cat_1.1500245489265888,ps_car_11_cat_1.1800478935086876,ps_car_11_cat_1.210071238090786,ps_car_11_cat_1.2400945826728849
count,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,200000.000000,2.000000e+05,2.000000e+05,2.000000e+05,...,200000.000000,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,-4.661160e-17,1.601563e-16,-4.462208e-17,4.348522e-17,6.785683e-17,-1.007550e-16,0.000000,2.046363e-17,-4.220624e-17,-6.252776e-18,...,0.005090,0.003105,0.00319,0.004255,0.016775,0.009200,0.012870,0.003385,0.035640,0.159235
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003,1.000003e+00,1.000003e+00,1.000003e+00,...,0.071163,0.055636,0.05639,0.065092,0.128428,0.095475,0.112714,0.058082,0.185391,0.365896
min,-9.905914e-01,-1.636898e+00,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-9.905914e-01,-9.067708e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-4.948532e-01,-1.766434e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.966230e-01,9.185477e-01,1.359482e+00,1.544515e+00,-4.620329e-01,-4.647157e-01,-0.021685,-4.391711e-02,-1.086872e-01,-3.303316e-02,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.479576e+00,2.378802e+00,1.359482e+00,1.544515e+00,2.164348e+00,2.151853e+00,46.115719,2.277017e+01,9.200712e+00,3.027261e+01,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [73]:
display_metrics(evaluate_model(X_train, y_train, X_test, y_test))

Accuracy: 0.5959
Precision: 0.6026
Recall: 0.5633
F1: 0.5823


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [74]:
CTR_data = data.copy()

cat_columns = []

for column in data:
    if "cat" in column:
        cat_columns.append(column)
        unique_vals = np.unique(data[column])
        for val in unique_vals:
            CTR_data[column][data[column] == val] = np.mean(target[data[column] == val])

X_train, X_test, y_train, y_test = train_test_split(normalize(CTR_data), target, test_size=0.5)
CTR_data.describe()

C:\Users\Егор\AppData\Local\Temp\ipykernel_6204\3777042462.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  CTR_data[column][data[column] == val] = np.mean(target[data[column] == val])
C:\Users\Егор\AppData\Local\Temp\ipykernel_6204\3777

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.500000,4.483870,0.500000,0.500000,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.015959,2.739255,0.020419,0.050252,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.494915,2.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.494915,4.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.500783,7.000000,0.517318,0.480547,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.911215,11.000000,0.964072,0.697423,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [75]:
display_metrics(evaluate_model(X_train, y_train, X_test, y_test))

Accuracy: 0.5945
Precision: 0.5978
Recall: 0.5733
F1: 0.5853


__Вывод:__

Качество
Качество между моделью обученной с использованием счетчиков (CTR) и OneHotEncoding не так сильно различается. Значения упали всего на ~0.15-0.3%, что не является критическим.

Время обучения
Время в свою очередь сильно изменилось. С целых 2 секунд до жалких 0.3 секунд. Время улучшилось в 6 раз

Итог
По сравнению с выигрышем времени проигрыш в точности никчемен, поэтому CTR показывает свое превосходство в данном случае над OneHotEncoding

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:

вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [76]:
CTR_noise_data = data.copy()

for col in categories:
    unique_vals = np.unique(data[col])
    for val in unique_vals:
        target_vals = target[data[col] == val]
        CTR_noise_data[col][data[col] == val] = np.mean(target_vals) + np.random.random(target_vals.shape) * 0.02

X_train, X_test, y_train, y_test = train_test_split(normalize(CTR_noise_data), target, test_size=0.5)
CTR_noise_data.describe()

C:\Users\Егор\AppData\Local\Temp\ipykernel_6204\1194169326.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  CTR_noise_data[col][data[col] == val] = np.mean(target_vals) + np.random.random(target_vals.shape) * 0.02
C:\Users\Егор\AppData\Lo

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.510005,4.483870,0.510011,0.510012,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.016993,2.739255,0.021214,0.050562,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.501811,2.000000,0.495028,0.486405,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.508331,4.000000,0.503795,0.492235,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.514139,7.000000,0.525746,0.498071,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.931036,11.000000,0.984071,0.717421,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
display_metrics(evaluate_model(X_train, y_train, X_test, y_test))

Accuracy: 0.5925
Precision: 0.5997
Recall: 0.5623
F1: 0.5804


In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def min_val(x, y):
    if x < y:
        return x
    return y

fold_data = data.copy()

def calc_out_of_fold_mean(index, cumulative_sum, fold_len, total_len):
    fold_index = index // fold_len
    left_idx = fold_index * fold_len
    right_idx = min_val(total_len, (fold_index + 1) * fold_len)
    full_sum = cumulative_sum[-1]
    fold_sum_val = cumulative_sum[right_idx] - cumulative_sum[left_idx]
    out_fold_sum = full_sum - fold_sum_val
    out_fold_count = total_len - (right_idx - left_idx)
    if out_fold_count == 0:
        return full_sum / total_len
    return out_fold_sum / out_fold_count

num_folds = 8

for col in categories:
    result = []
    unique_vals = np.unique(data[col])
    cumulative_sums = {}
    counters = {}

    for val in unique_vals:
        target_vals = np.array(target[data[col] == val])
        cumulative_sums[val] = np.hstack(([0], np.cumsum(target_vals)))

    for curr_val in data[col]:
        if curr_val not in counters:
            counters[curr_val] = 0
        total_count = len(cumulative_sums[curr_val]) - 1
        fold_length = (total_count + num_folds - 1) // num_folds
        result.append(calc_out_of_fold_mean(counters[curr_val], cumulative_sums[curr_val], fold_length, total_count))
        counters[curr_val] += 1

    fold_data[col] = result

X_train, X_test, y_train, y_test = train_test_split(normalize(fold_data), target, test_size=0.5)
fold_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.499997,4.483870,0.499997,0.499984,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.072264,2.739255,0.072364,0.085039,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.422759,0.000000,0.412820,0.406339,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.422759,2.000000,0.428571,0.406339,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.565613,4.000000,0.555677,0.549196,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.565618,7.000000,0.555677,0.549196,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,1.000000,11.000000,1.000000,0.797091,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [79]:
display_metrics(evaluate_model(X_train, y_train, X_test, y_test))

Accuracy: 0.9997
Precision: 0.9997
Recall: 0.9997
F1: 0.9997


__Вывод:__